In [1]:
import math
import numpy as np

# Elliptical, parabolic, and hyperbolic orbits using the Euler method

In the simple_two_body.ipynb notebook I implemented circular orbits with arbitary starting positions and confirmed that the truncation error converged linearly with timestep for the Euler method.  

In this notebook I plan to implement hyperbolic, elliptical, and parabolic orbits using the Euler method. Parabolic orbits are marginally bound, hyperbolic orbits are unbound, and elliptical orbits are bound but oblong. All orbits should have the center of mass at one focus for both stars orbit and be symmetric between the two stars. 

Elliptical orbits should be elongated away from the other star. 

Hyperbolic and parabolic orbits should open away from the other star. 

All orbits should conserve energy and angular momentum. Later this could be enforced. I will probably not do that in this notebook. 

It would be meaningless to evaluate the L0 error for a hyperbolic or parabolic orbit since there is no cycle and the L0 error is found after one or more cycles when the system returns to its starting point. However, it would be possible to evaluate it for the elliptical orbit system, and I intend to check that the convergence of the L0 error with dt is linear as it should be for the Euler method. 

Because the convergence can be tested for ellipses only and because ellipses are closed so make it clear if they are not working correctly graphically, I will begin with ellipses then proceed to parabolas and hyperbolas. 

# Conic sections

Circles, ellipses, parabolas, and hyperbolas are called conic sections because they can all be obtained by taking slices of an infinite cone. 

Conic sections are related to each other through a parameter called eccentricity. Circles have an eccentricity of 0. Ellipses have an eccentricity of 0<e<1. Parabolas have an eccentricity of e=1. Hyperbolas have an eccentricity of e>1. 

Circles are described by the equation x^2+y^2=r^2 where r is the radius. They can also be described parametrically using sine and cosine. That is done below. 

In [2]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
radius=10
t=2*math.pi*0.01*np.arange(103)
centerx=3
centery=5
p = figure(title="Circle", plot_width=400, plot_height=400)
p.line(centerx+np.cos(t),centery+np.sin(t),line_width=3,line_color="blue",legend="Circle")
p.circle(centerx,centery,line_width=15,line_color="green",legend="Center")
p.legend.location="bottom_right"
show(p)

Loading BokehJS ...

Ellipses are shaped like ovals. They have two locations called focuses that determine their shapes. If a string were connected to both focuses, which are inside the ellipse, you could slide a pencil along the string and draw the ellipse keeping the string taut. The two focuses and the ellipse form a triangle with hypoteneuses of fixed length. 

Half the shorter of the two widths of the ellipse is called the semimajor axis, a. It's like a radius for ellipses. The lenght of the longer width is (1+e)*a. The focus is offset from the center by f=e*sqrt(1-e^2)*a. 

Ellipses can also be drawn at angles to the axis. An ellipse aligned with the x and y axis is shown below. 

The nonparametric equation for an ellipse is x^2/a^2+y^2/b^2=1
Below the ellipse is described parametrically. 

In [3]:
def getfocus(a,ecc):
    b=a*(1+ecc)
    return np.sqrt(b**2-a**2)

In [4]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
radius=10
t=2*math.pi*0.01*np.arange(103)
a=10
ecc=.5
b=a*(1+ecc)
centerx=0
centery=0
focusx=centerx+getfocus(a,ecc)
focusy=centery+0.0
p = figure(title="Ellipse", plot_width=400, plot_height=400, x_range=[-15,15], y_range=[-15,15])
p.line(centerx+b*np.cos(t),centery+a*np.sin(t),line_width=3,line_color="blue",legend="Circle")
p.circle(focusx,focusy,line_width=15,line_color="green",legend="Focus 1")
p.circle(centerx-getfocus(a,ecc),focusy, line_width=15, line_color="red", legend="Focus 2")
p.legend.location="bottom_right"
show(p)

Loading BokehJS ...

In [5]:
import numpy as np
import matplotlib.pyplot as plt

def NewtonianForce(mass1,mass2, r):
    return (6.67408*10**-11)*mass1*mass2/r/r;

In [6]:
def euler(h,t, x,y,z,f,debugprint):
    xstep = (f(t, x, y,z)+f(t+h,x+h,y,z))/2*h
    xnew=x+xstep
    if(debugprint):
        print(h,t,x,xstep,xnew)
    return t+h,xnew

In [7]:
import math
def InitialDataEqualMassConic(radius,ecc,angle,initmass):
    orbitalangle=angle
    print("angle",angle)
    phi=np.array([math.pi+orbitalangle,orbitalangle])
    orbitalradius=radius*np.ones(2) #semimajor axis
    eccentricity=ecc
    print(eccentricity)
    mass=np.ones(2)
    masses=initmass*mass #*masssun (natural units)
    return phi,orbitalangle,orbitalradius,eccentricity, masses
    

In [8]:
import random,numpy as np
rad0=100
ecc0=0.3
angle0=0.0
mass0=1.0
initdateqellipse=InitialDataEqualMassConic(rad0,ecc0,angle0,mass0)
print(initdateqellipse)

angle 0.0
0.3
(array([3.14159265, 0.        ]), 0.0, array([100., 100.]), 0.3, array([1., 1.]))


In [47]:
def getxyuveqtwoellipses(initdat):
    phi,orbitangle,orbitalradius,eccentricity, masses=initdat
    print("initdat", orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=1
    Gconstant=1
    #fix x0 y0 at one star, disregard initial data, use orbital radius as separation between stars
    #this is consistant with choice in previous part
    cosphi=np.cos(phi)
    sinphi=np.sin(phi)
    coordsep=orbitalradius #/2.
    print("coordsep",coordsep)
    #x0=orbitalradius/2.*np.cos(phi)*metersperAU
    #y0=orbitalradius/2.*np.sin(phi)*metersperAU
    count=0
    for phi0 in phi:
        if phi0==0:
            print("zero")
            #x0[count]=orbitalradius[count]/2.
            #y0[count]=0
            cosphi[count]=1.0
            sinphi[count]=0.0
        if phi0==math.pi:
            cosphi[count]=-1.0
            sinphi[count]=0.0
            print("pi")
            #x0[count]=-orbitalradius[count]/2.
            #y0[count]=0
        if phi0==math.pi/2.:
            cosphi[count]=0.0
            sinphi[count]=1.0
            #x0[count]=0
            #y0[count]=orbitalradius[count]/2.
        if phi0==3*math.pi/2.:
            cosphi[count]=0.0
            sinphi[count]=-1.0
            #x0[count]=0
            #y0[count]=-orbitalradius[count]/2.
        count+=1
    x0=coordsep*cosphi
    y0=coordsep*sinphi
    #x0[1]=0.0
    #y0[1]=0.0
    z0=np.zeros(2)
    print(x0)
    print(y0)
    v=np.zeros(2)
    a=np.zeros(2)
    ux0=0.
    uy0=0.
    uz0=0.
    ax0=0.
    ay0=0.
    az0=0.
    vx0=0.
    vy0=0.
    vz0=0.
    print(eccentricity)
    #masses=np.zeros(2)
    #masses[0]=masses[1]+masses[0]
    #masses[1]=(masses[1]*masses[0])/(masses[1]+masses[0])
    if eccentricity<1 and eccentricity>0:
        #elliptical
        orbitalr=orbitalradius[0]/2. #reduced mass
        focusdisp=getfocus(orbitalr,eccentricity) #displacement of the center due to the focus being at the center
        coordsep=orbitalr*(1+eccentricity) #start at aphelion
        starsep=2.*coordsep #two stars, at opposite ends of the orbit
        Fapastron=masses[1]*masses[0]/starsep**2
        x0=(coordsep+focusdisp)*cosphi
        y0=(coordsep+focusdisp)*sinphi
        x0[0]=0.0
        y0[0]=0.0
        v=np.zeros(2)
        v=np.sqrt(masses[1]*masses[0]*(2./starsep-1./(2.*orbitalr)/masses)) 
        ux0=-v[0]*sinphi
        uy0=v[1]*cosphi #initial data in y only 
        uz0=np.zeros(2)
    
        a=np.zeros(2)
        a=Fapastron/masses
    
        ax0=-a[0]*cosphi
        ay0=-a[1]*sinphi
        az0=np.zeros(2)
    elif eccentricity[0]==0.0: #circular
        #start at perihelion for both (eliptical, doesn't generalize to three body)
        #actually start with circular orbit
        ux0=np.zeros(2) #*149597870700
        #centrepital force balances gravitational force
        metersperAU=1 #natural units
        #G=1
        Gconstant=1
        #Fcentripital=mass1*v**2/rphys
        #centF=accel
        #Faccel=G*m1*m2/r^2
        r0=2.*orbitalradius #Mystery factor of 2
        print("r0", r0)
        v=np.zeros(2)
        for i in np.arange(2):
            v[i]=np.sqrt(Gconstant*masses[(i+1)%2]/np.abs(r0[i]))
        print(v)
        #r0=orbitalradius #np.sqrt(x0**2+y0**2)
        ux0=-v[0]*sinphi
        uy0=v[1]*cosphi #initial data in y only 
        uz0=np.zeros(2)
    
        a=np.zeros(2)
        a=Fapastron/masses
    
        ax0=-a[0]*cosphi
        ay0=-a[1]*sinphi
        az0=np.zeros(2)

    
    
    return masses,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0

In [48]:
xyuvaeqtwoellipses=getxyuveqtwoellipses(initdateqellipse)
print(xyuvaeqtwoellipses)

initdat [100. 100.] [3.14159265 0.        ] [-1.  1.] [1.2246468e-16 0.0000000e+00]
coordsep [100. 100.]
pi
zero
[-100.  100.]
[0. 0.]
0.3
(array([1., 1.]), array([  0.        , 106.53311931]), array([0., 0.]), array([0., 0.]), array([-0., -0.]), array([-0.07337994,  0.07337994]), array([0., 0.]), array([ 5.91715976e-05, -5.91715976e-05]), array([-0., -0.]), array([0., 0.]))


In [51]:
def timestep(step,t,dt,mtotal, mass,xi,yi,zi, vxi, vyi, vzi, axi, ayi, azi):
    xii=np.zeros(np.size(xi))
    vxii=np.zeros(np.size(vxi))
    yii=np.zeros(np.size(yi))
    vyii=np.zeros(np.size(vyi))
    zii=np.zeros(np.size(vzi))
    vzii=np.zeros(np.size(vzi))
    rii=np.zeros(np.size(xi))
    axii=np.zeros(np.size(axi))
    ayii=np.zeros(np.size(ayi))
    azii=np.zeros(np.size(azi))
    
    #This is redundant in the center of mass system
    Gconstant=6.408*10**-11
    Gconstant=1
    for k in np.arange(len(rii)):
        for j in np.arange(len(rii)):
            if j!=k:
                rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                #print(rreljk)
                axii[j]-=Gconstant*mass[k]*(xi[j]  - xi[k])/rreljk**3
                ayii[j]-=Gconstant*mass[k]*(yi[j]  - yi[k])/rreljk**3
                azii[j]-=Gconstant*mass[k]*(zi[j]  - zi[k])/rreljk**3
    #print(xii)
    
    for m in np.arange(len(x)):
        #m represents choices of mass
        i=step
        
        xii[m] = xi[m] + dt*vxi[m]
        vxii[m] = vxi[m] + dt*axi[m]
        yii[m]= yi[m] + dt*vyi[m]
        vyii[m] = vyi[m] + dt*ayi[m]
        zii[m]= zi[m] + dt*vzi[m]
        vzii[m] = vzi[m] + dt*azi[m]
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
        
        #print(vyii[m],vyi[m],ayi[m],dt)
        #print(yii[m],yi[m],vyi[m],dt)
    
    
    return mass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii
                    

In [52]:
dt=1 #*31556926 #seconds per year
numsteps=10000
mass0,x,y,z0,vx,vy,vz0,ax,ay,az0=xyuvaeqtwoellipses
print(x)
xcoord1=[]
xcoord2=[]
ycoord1=[]
ycoord2=[]
t=0.0
masstotal=1.
for i in np.arange(1,numsteps):
    mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(i,t,dt,masstotal,mass0,x,y,z0,vx,vy,vz0,ax,ay,az0)
    print(x,y,vx,vy,ax,ay)
    #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
    #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
    #print(ax)
    xcoord1.append(x[0])
    xcoord2.append(x[1])
    ycoord1.append(y[0])
    ycoord2.append(y[1])
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(2,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(3,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(4,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 

[  0.         106.53311931]
[  0.         106.53311931] [-0.07337994  0.07337994] [ 5.91715976e-05 -5.91715976e-05] [-0.07337994  0.07337994] [ 8.81111183e-05 -8.81111183e-05] [0. 0.]
[5.91715976e-05 1.06533060e+02] [-0.14675988  0.14675988] [ 0.00014728 -0.00014728] [-0.07337994  0.07337994] [ 8.81108674e-05 -8.81108674e-05] [ 1.21381409e-07 -1.21381409e-07]
[2.06454314e-04 1.06532913e+02] [-0.22013982  0.22013982] [ 0.00023539 -0.00023539] [-0.07337982  0.07337982] [ 8.81103107e-05 -8.81103107e-05] [ 2.42761553e-07 -2.42761553e-07]
[4.41847897e-04 1.06532677e+02] [-0.29351963  0.29351963] [ 0.0003235 -0.0003235] [-0.07337957  0.07337957] [ 8.81095439e-05 -8.81095439e-05] [ 3.64140167e-07 -3.64140167e-07]
[7.65351791e-04 1.06532354e+02] [-0.36689921  0.36689921] [ 0.00041161 -0.00041161] [-0.07337921  0.07337921] [ 8.81085669e-05 -8.81085669e-05] [ 4.85516784e-07 -4.85516784e-07]
[1.17696523e-03 1.06531942e+02] [-0.44027842  0.44027842] [ 0.00049972 -0.00049972] [-0.07337872  0.073378

[  1.09674475 105.43637457] [-11.58950863  11.58950863] [ 0.01384198 -0.01384198] [-0.07188671  0.07188671] [ 8.54843504e-05 -8.54843504e-05] [ 1.88675248e-05 -1.88675248e-05]
[  1.11058673 105.42253259] [-11.66139534  11.66139534] [ 0.01392747 -0.01392747] [-0.07186784  0.07186784] [ 8.54516717e-05 -8.54516717e-05] [ 1.89830631e-05 -1.89830631e-05]
[  1.1245142  105.40860512] [-11.73326318  11.73326318] [ 0.01401292 -0.01401292] [-0.07184886  0.07184886] [ 8.54187992e-05 -8.54187992e-05] [ 1.90985295e-05 -1.90985295e-05]
[  1.13852712 105.3945922 ] [-11.80511204  11.80511204] [ 0.01409834 -0.01409834] [-0.07182976  0.07182976] [ 8.53857332e-05 -8.53857332e-05] [ 1.92139236e-05 -1.92139236e-05]
[  1.15262545 105.38049386] [-11.8769418  11.8769418] [ 0.01418372 -0.01418372] [-0.07181055  0.07181055] [ 8.53524737e-05 -8.53524737e-05] [ 1.93292451e-05 -1.93292451e-05]
[  1.16680918 105.36631014] [-11.94875235  11.94875235] [ 0.01426908 -0.01426908] [-0.07179122  0.07179122] [ 8.53190211e-

[  4.34511993 102.18799939] [-22.70870472  22.70870472] [ 0.02690209 -0.02690209] [-0.06749269  0.06749269] [ 7.8011979e-05 -7.8011979e-05] [ 3.60846905e-05 -3.60846905e-05]
[  4.37202201 102.1610973 ] [-22.77619741  22.77619741] [ 0.0269801 -0.0269801] [-0.0674566  0.0674566] [ 7.79516528e-05 -7.79516528e-05] [ 3.61841572e-05 -3.61841572e-05]
[  4.39900211 102.1341172 ] [-22.84365401  22.84365401] [ 0.02705805 -0.02705805] [-0.06742042  0.06742042] [ 7.78911766e-05 -7.78911766e-05] [ 3.62834971e-05 -3.62834971e-05]
[  4.42606017 102.10705915] [-22.91107443  22.91107443] [ 0.02713594 -0.02713594] [-0.06738414  0.06738414] [ 7.78305505e-05 -7.78305505e-05] [ 3.63827099e-05 -3.63827099e-05]
[  4.45319611 102.0799232 ] [-22.97845857  22.97845857] [ 0.02721377 -0.02721377] [-0.06734775  0.06734775] [ 7.7769775e-05 -7.7769775e-05] [ 3.64817953e-05 -3.64817953e-05]
[  4.48040988 102.05270943] [-23.04580632  23.04580632] [ 0.02729154 -0.02729154] [-0.06731127  0.06731127] [ 7.77088504e-05 -7.

[ 9.82984044 96.70327888] [-33.34795045  33.34795045] [ 0.03892418 -0.03892418] [-0.0602425  0.0602425] [ 6.6205761e-05 -6.6205761e-05] [ 5.06913268e-05 -5.06913268e-05]
[ 9.86876462 96.6643547 ] [-33.40819295  33.40819295] [ 0.03899039 -0.03899039] [-0.06019181  0.06019181] [ 6.61253987e-05 -6.61253987e-05] [ 5.07668755e-05 -5.07668755e-05]
[ 9.90775501 96.62536431] [-33.46838477  33.46838477] [ 0.03905651 -0.03905651] [-0.06014105  0.06014105] [ 6.60449465e-05 -6.60449465e-05] [ 5.08422677e-05 -5.08422677e-05]
[ 9.94681152 96.58630779] [-33.52852581  33.52852581] [ 0.03912256 -0.03912256] [-0.0600902  0.0600902] [ 6.59644048e-05 -6.59644048e-05] [ 5.09175034e-05 -5.09175034e-05]
[ 9.98593408 96.54718523] [-33.58861602  33.58861602] [ 0.03918852 -0.03918852] [-0.06003929  0.06003929] [ 6.58837741e-05 -6.58837741e-05] [ 5.09925823e-05 -5.09925823e-05]
[10.02512261 96.50799671] [-33.6486553  33.6486553] [ 0.03925441 -0.03925441] [-0.05998829  0.05998829] [ 6.58030547e-05 -6.58030547e-05

[17.18416306 89.34895625] [-42.65317623  42.65317623] [ 0.04880973 -0.04880973] [-0.05085095  0.05085095] [ 5.18207482e-05 -5.18207482e-05] [ 6.1101873e-05 -6.1101873e-05]
[17.23297279 89.30014652] [-42.70402718  42.70402718] [ 0.04886155 -0.04886155] [-0.05078985  0.05078985] [ 5.17304366e-05 -5.17304366e-05] [ 6.11508003e-05 -6.11508003e-05]
[17.28183434 89.25128498] [-42.75481702  42.75481702] [ 0.04891328 -0.04891328] [-0.05072869  0.05072869] [ 5.16400947e-05 -5.16400947e-05] [ 6.11995696e-05 -6.11995696e-05]
[17.33074762 89.2023717 ] [-42.80554572  42.80554572] [ 0.04896492 -0.04896492] [-0.0506675  0.0506675] [ 5.15497227e-05 -5.15497227e-05] [ 6.12481808e-05 -6.12481808e-05]
[17.37971254 89.15340678] [-42.85621321  42.85621321] [ 0.04901647 -0.04901647] [-0.05060625  0.05060625] [ 5.14593211e-05 -5.14593211e-05] [ 6.12966341e-05 -6.12966341e-05]
[17.42872901 89.10439031] [-42.90681946  42.90681946] [ 0.04906793 -0.04906793] [-0.05054495  0.05054495] [ 5.136889e-05 -5.136889e-05

...

[126.75262565 -20.21950634] [-12.56000626  12.56000626] [ 0.01100234 -0.01100234] [ 0.05232379 -0.05232379] [-4.43346239e-05  4.43346239e-05] [ 7.61024054e-06 -7.61024054e-06]
[126.763628   -20.23050868] [-12.50768246  12.50768246] [ 0.01095801 -0.01095801] [ 0.0523314 -0.0523314] [-4.43376215e-05  4.43376215e-05] [ 7.5780462e-06 -7.5780462e-06]
[126.774586   -20.24146669] [-12.45535106  12.45535106] [ 0.01091367 -0.01091367] [ 0.05233898 -0.05233898] [-4.43406035e-05  4.43406035e-05] [ 7.54585457e-06 -7.54585457e-06]
[126.78549968 -20.25238036] [-12.40301208  12.40301208] [ 0.01086933 -0.01086933] [ 0.05234653 -0.05234653] [-4.43435701e-05  4.43435701e-05] [ 7.51366566e-06 -7.51366566e-06]
[126.79636901 -20.26324969] [-12.35066555  12.35066555] [ 0.01082499 -0.01082499] [ 0.05235404 -0.05235404] [-4.43465212e-05  4.43465212e-05] [ 7.48147944e-06 -7.48147944e-06]
[126.80719399 -20.27407468] [-12.29831151  12.29831151] [ 0.01078064 -0.01078064] [ 0.05236152 -0.05236152] [-4.43494568e-05

[127.83089529 -21.29777598] [-5.49339317  5.49339317] [ 0.00504078 -0.00504078] [ 0.05305766 -0.05305766] [-4.46007238e-05  4.46007238e-05] [ 3.31783878e-06 -3.31783878e-06]
[127.83593607 -21.30281676] [-5.4403355  5.4403355] [ 0.00499618 -0.00499618] [ 0.05306098 -0.05306098] [-4.46016996e-05  4.46016996e-05] [ 3.28594991e-06 -3.28594991e-06]
[127.84093226 -21.30781294] [-5.38727452  5.38727452] [ 0.00495158 -0.00495158] [ 0.05306427 -0.05306427] [-4.46026606e-05  4.46026606e-05] [ 3.25406286e-06 -3.25406286e-06]
[127.84588384 -21.31276452] [-5.33421025  5.33421025] [ 0.00490698 -0.00490698] [ 0.05306752 -0.05306752] [-4.46036069e-05  4.46036069e-05] [ 3.22217763e-06 -3.22217763e-06]
[127.85079082 -21.3176715 ] [-5.28114273  5.28114273] [ 0.00486238 -0.00486238] [ 0.05307074 -0.05307074] [-4.46045384e-05  4.46045384e-05] [ 3.19029421e-06 -3.19029421e-06]
[127.85565319 -21.32253388] [-5.22807199  5.22807199] [ 0.00481777 -0.00481777] [ 0.05307393 -0.05307393] [-4.46054552e-05  4.460545

[128.09452159 -21.56140227] [ 2.27197262 -2.27197262] [-0.00147381  0.00147381] [ 0.05320533 -0.05320533] [-4.45885245e-05  4.45885245e-05] [-1.32209418e-06  1.32209418e-06]
[128.09304778 -21.55992846] [ 2.32517795 -2.32517795] [-0.0015184  0.0015184] [ 0.05320401 -0.05320401] [-4.4587374e-05  4.4587374e-05] [-1.35378928e-06  1.35378928e-06]
[128.09152938 -21.55841006] [ 2.37838195 -2.37838195] [-0.00156299  0.00156299] [ 0.05320265 -0.05320265] [-4.45862091e-05  4.45862091e-05] [-1.38548357e-06  1.38548357e-06]
[128.08996639 -21.55684707] [ 2.43158461 -2.43158461] [-0.00160757  0.00160757] [ 0.05320127 -0.05320127] [-4.45850297e-05  4.45850297e-05] [-1.41717705e-06  1.41717705e-06]
[128.08835882 -21.5552395 ] [ 2.48478587 -2.48478587] [-0.00165216  0.00165216] [ 0.05319985 -0.05319985] [-4.45838358e-05  4.45838358e-05] [-1.44886973e-06  1.44886973e-06]
[128.08670666 -21.55358734] [ 2.53798572 -2.53798572] [-0.00169674  0.00169674] [ 0.0531984 -0.0531984] [-4.45826274e-05  4.45826274e-

[127.43963715 -20.90651783] [ 9.79909659 -9.79909659] [-0.0077871  0.0077871] [ 0.05270056 -0.05270056] [-4.42799556e-05  4.42799556e-05] [-5.81780743e-06  5.81780743e-06]
[127.43185005 -20.89873074] [ 9.85179716 -9.85179716] [-0.00783138  0.00783138] [ 0.05269474 -0.05269474] [-4.42767398e-05  4.42767398e-05] [-5.84945461e-06  5.84945461e-06]
[127.42401867 -20.89089936] [ 9.9044919 -9.9044919] [-0.00787565  0.00787565] [ 0.05268889 -0.05268889] [-4.42735092e-05  4.42735092e-05] [-5.88110193e-06  5.88110193e-06]
[127.41614302 -20.88302371] [ 9.95718079 -9.95718079] [-0.00791993  0.00791993] [ 0.05268301 -0.05268301] [-4.4270264e-05  4.4270264e-05] [-5.91274939e-06  5.91274939e-06]
[127.40822309 -20.87510378] [ 10.00986381 -10.00986381] [-0.0079642  0.0079642] [ 0.0526771 -0.0526771] [-4.4267004e-05  4.4267004e-05] [-5.94439702e-06  5.94439702e-06]
[127.4002589  -20.86713958] [ 10.06254091 -10.06254091] [-0.00800846  0.00800846] [ 0.05267116 -0.05267116] [-4.42637293e-05  4.42637293e-05

[126.09761712 -19.56449781] [ 16.4349662 -16.4349662] [-0.01337997  0.01337997] [ 0.05170838 -0.05170838] [-4.37524081e-05  4.37524081e-05] [-9.84024324e-06  9.84024324e-06]
[126.08423715 -19.55111783] [ 16.48667457 -16.48667457] [-0.01342373  0.01342373] [ 0.05169854 -0.05169854] [-4.37472873e-05  4.37472873e-05] [-9.87195865e-06  9.87195865e-06]
[126.07081342 -19.53769411] [ 16.53837311 -16.53837311] [-0.01346747  0.01346747] [ 0.05168867 -0.05168867] [-4.37421512e-05  4.37421512e-05] [-9.90367499e-06  9.90367499e-06]
[126.05734595 -19.52422663] [ 16.59006178 -16.59006178] [-0.01351122  0.01351122] [ 0.05167876 -0.05167876] [-4.37369997e-05  4.37369997e-05] [-9.93539226e-06  9.93539226e-06]
[126.04383473 -19.51071542] [ 16.64174054 -16.64174054] [-0.01355495  0.01355495] [ 0.05166883 -0.05166883] [-4.37318328e-05  4.37318328e-05] [-9.96711046e-06  9.96711046e-06]
[126.03027978 -19.49716046] [ 16.69340937 -16.69340937] [-0.01359868  0.01359868] [ 0.05165886 -0.05165886] [-4.37266505e-

In [59]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="dt=1, Euler Method, Two ellipse produce two circles, e=0.5", plot_width=400, plot_height=400, y_range=[-150,150],x_range=[-150,150])
t = np.linspace(0, 2*math.pi, 100)
focusx=getfocus(0.5*rad0,ecc0)
b=0.5*rad0*(1+ecc)
print(b)
print(focusx)
p.triangle(xcoord1,ycoord1, size=1, line_color="purple",fill_color="purple", fill_alpha=0.5, legend="Ellipse 1")
p.triangle(xcoord2,ycoord2, size=1, line_color="blue", fill_color="blue", fill_alpha=0.5,legend="Ellipse 2")
p.line(-focusx+rad0*(1+ecc0)*np.cos(t),rad0*np.sin(t), line_color="green", legend="Ideal ellipse")
p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p) # show the results

Loading BokehJS ...

75.0
41.53311931459037


I don't know why it's producing circles offset from eachother, rather than ellipses with eccentricities of 0.5.